In [ ]:
import torch
from ultralytics import YOLO
import os
import numpy as np
import cv2

# YOLOv8-pose 모델 로드
model = YOLO('yolov8n-pose.pt')  # Pre-trained YOLOv8-pose 모델 사용

# 경로 설정
image_folder = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/aegis_pose_data/violence_B_30fps_flipped"  # 입력 이미지 경로
output_folder = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/filtered_keypoints_flipped"  # 결과 저장 경로
os.makedirs(output_folder, exist_ok=True)

# 키포인트 인덱스 (COCO 기준)
KEYPOINTS_INDEX = {
    "left_shoulder": 5, "left_elbow": 7, "left_wrist": 9,
    "right_shoulder": 6, "right_elbow": 8, "right_wrist": 10
}

# 바닥 키포인트 필터링 및 유효 키포인트 추출
def filter_keypoints(keypoints, confidences, image_height):
    """
    바닥 키포인트 제거 (하단 30%) 및 신뢰도 높은 키포인트만 필터링
    """
    filtered_keypoints = {}
    for name, idx in KEYPOINTS_INDEX.items():
        if idx < len(keypoints):
            x, y = keypoints[idx]
            conf = confidences[idx]

            # 조건: 신뢰도가 높고 바닥(하단 30%)에 위치하지 않은 키포인트만 저장
            if conf > 0.5 and y < image_height * 0.7:  # 하단 30% 필터링
                filtered_keypoints[name] = (int(x), int(y))
    return filtered_keypoints

# 이미지에서 키포인트 추출 및 필터링 (여러 사람 처리)
def extract_keypoints(image_path):
    image = cv2.imread(image_path)  # OpenCV로 이미지 불러오기
    results = model(image, verbose=False)  # YOLOv8-pose 실행

    people_keypoints = []  # 여러 사람의 키포인트 저장 리스트

    for result in results[0].keypoints:
        if result.xy is not None:
            keypoints = result.xy[0].cpu().numpy()  # 키포인트 좌표
            confidences = result.conf[0].cpu().numpy()  # 키포인트 신뢰도
            image_height = image.shape[0]

            # 유효 키포인트 필터링 적용
            filtered_keypoints = filter_keypoints(keypoints, confidences, image_height)
            if filtered_keypoints:  # 유효 키포인트가 존재하는 경우만 저장
                people_keypoints.append(filtered_keypoints)

    return people_keypoints

# 전체 이미지 처리
valid_keypoints = {}
for image_file in sorted(os.listdir(image_folder)):
    if image_file.lower().endswith(('.jpg', '.png')):
        image_path = os.path.join(image_folder, image_file)
        people_keypoints = extract_keypoints(image_path)
        if people_keypoints:  # 유효 키포인트가 존재하면 저장
            valid_keypoints[image_file] = people_keypoints
        print(f"{image_file}: {'Keypoints Extracted' if people_keypoints else 'No Valid Keypoints'}")

# 결과 저장
output_file = os.path.join(output_folder, "valid_keypoints_multiple.npy")
np.save(output_file, valid_keypoints)
print("Valid keypoints extraction completed and saved to 'valid_keypoints_multiple.npy'.")


### 키포인트 추출 이미지로 시각화

In [ ]:
import numpy as np
import os
import cv2

# 경로 설정
keypoints_file = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/filtered_keypoints_flipped/valid_keypoints_multiple.npy"  # 키포인트 파일 경로
image_folder = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/aegis_pose_data/violence_B_30fps_flipped"  # 원본 이미지 폴더
output_folder = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/keypoint_flipped"  # 결과 이미지 저장 폴더
os.makedirs(output_folder, exist_ok=True)

# 키포인트 데이터 불러오기
keypoints_data = np.load(keypoints_file, allow_pickle=True).item()

# 키포인트 시각화 함수
def visualize_keypoints(image_path, keypoints_list, output_path):
    image = cv2.imread(image_path)
    colors = [(0, 0, 255), (0, 255, 0), (255, 0, 0), (255, 255, 0)]  # 각 사람의 키포인트 색상

    for idx, keypoints in enumerate(keypoints_list):  # 여러 사람 순회
        color = colors[idx % len(colors)]  # 색상 선택 (순환)
        for name, (x, y) in keypoints.items():
            cv2.circle(image, (x, y), 5, color, -1)  # 키포인트 그리기
            cv2.putText(image, name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)  # 라벨 표시
    
    cv2.imwrite(output_path, image)  # 결과 이미지 저장

# 전체 이미지 처리 및 시각화
for image_file, keypoints_list in keypoints_data.items():
    image_path = os.path.join(image_folder, image_file)
    output_path = os.path.join(output_folder, image_file)
    if os.path.exists(image_path) and keypoints_list:  # 이미지가 존재하고 키포인트가 있으면
        visualize_keypoints(image_path, keypoints_list, output_path)
        print(f"Processed: {image_file}")
    else:
        print(f"Skipped: {image_file}")

print(f"Visualization complete. Images saved in {output_folder}")


### 좌표변화량 계산

In [ ]:
import numpy as np
import os

# 경로 설정
keypoints_file = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/filtered_keypoints/valid_keypoints_multiple.npy"
output_file = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/coordinate_changes.npy"

# 데이터 로드
keypoints_data = np.load(keypoints_file, allow_pickle=True).item()

# 좌표 변화량 계산 함수
def calculate_coordinate_changes(data):
    previous_coords = {}
    changes = {}

    for frame, people in sorted(data.items()):
        frame_changes = []
        for person in people:  # 여러 사람의 키포인트
            person_changes = []
            for key in ["left_shoulder", "left_elbow", "left_wrist", "right_shoulder", "right_elbow", "right_wrist"]:
                current_coords = person.get(key, (0, 0))
                previous_coords.setdefault(key, current_coords)
                delta = abs(current_coords[0] - previous_coords[key][0]) + abs(current_coords[1] - previous_coords[key][1])
                person_changes.append(delta)
                previous_coords[key] = current_coords
            frame_changes.append(person_changes)
        changes[frame] = frame_changes
    return changes

# 변화량 계산
coordinate_changes = calculate_coordinate_changes(keypoints_data)

# 결과 저장
np.save(output_file, coordinate_changes)
print(f"Coordinate changes saved to: {output_file}")


### 좌표 변화량 시각화

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 경로 설정
coordinate_changes_file = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/coordinate_changes.npy"

# 변화량 데이터 불러오기
coordinate_changes = np.load(coordinate_changes_file, allow_pickle=True).item()

# 프레임 순서대로 정렬된 데이터 준비
frames = sorted(coordinate_changes.keys())  # 프레임 이름 정렬
keypoints_names = ["left_shoulder", "left_elbow", "left_wrist", "right_shoulder", "right_elbow", "right_wrist"]

# 변화량 시각화
def plot_average_coordinate_changes(coordinate_changes):
    plt.figure(figsize=(12, 8))

    # 각 키포인트별로 변화량 추출 및 평균 시각화
    for i, keypoint_name in enumerate(keypoints_names):
        average_changes = []  # 프레임당 평균 변화량 저장

        for frame in frames:
            people_changes = coordinate_changes[frame]  # 각 프레임의 모든 사람 변화량
            keypoint_sum = 0
            keypoint_count = 0

            for person_changes in people_changes:
                if len(person_changes) == 6:  # 어깨, 팔꿈치, 손목 변화량
                    keypoint_sum += person_changes[i]
                    keypoint_count += 1

            # 평균 변화량 계산
            avg_change = keypoint_sum / keypoint_count if keypoint_count > 0 else 0
            average_changes.append(avg_change)

        # 그래프 그리기
        plt.plot(range(len(frames)), average_changes, label=f"{keypoint_name}_avg")

    plt.title("Average Keypoint Coordinate Changes Over Frames")
    plt.xlabel("Frame Index")
    plt.ylabel("Average Coordinate Changes")
    plt.legend()
    plt.grid(True)
    plt.show()

# 변화량 시각화 실행
plot_average_coordinate_changes(coordinate_changes)


### 왼쪽 온리 키포인트

In [ ]:
import torch
from ultralytics import YOLO
import os
import numpy as np
import cv2

# YOLOv8-pose 모델 로드
model = YOLO('yolov8n-pose.pt')

# 경로 설정
image_folder = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/aegis_pose_data/violence_B_30fps_flipped"
output_folder = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/filtered_keypoints_left_only"
os.makedirs(output_folder, exist_ok=True)

# 키포인트 인덱스 (COCO 기준)
KEYPOINTS_INDEX = {
    "left_shoulder": 5, "left_elbow": 7, "left_wrist": 9,
    "right_shoulder": 6, "right_elbow": 8, "right_wrist": 10
}

# 바닥 키포인트 필터링 및 왼쪽 사람 선택
def filter_keypoints_and_select_leftmost(results, image_height):
    leftmost_keypoints = {}
    min_x = float('inf')  # x좌표 최소값 초기화

    for result in results.keypoints:  # 모든 사람의 키포인트 순회
        if result.xy is not None:
            keypoints = result.xy[0].cpu().numpy()
            confidences = result.conf[0].cpu().numpy()
            filtered_keypoints = {}

            # 키포인트 필터링 (신뢰도 & 바닥 제거)
            for name, idx in KEYPOINTS_INDEX.items():
                x, y = keypoints[idx]
                conf = confidences[idx]
                if conf > 0.7 and y < image_height * 0.7:  # 신뢰도와 바닥 필터링
                    filtered_keypoints[name] = (int(x), int(y))

            # 왼쪽 사람 선택 (x좌표가 가장 작은 사람)
            if filtered_keypoints:
                avg_x = sum(x for x, _ in filtered_keypoints.values()) / len(filtered_keypoints)
                if avg_x < min_x:
                    min_x = avg_x
                    leftmost_keypoints = filtered_keypoints

    return leftmost_keypoints

# 이미지 처리 및 왼쪽 사람 키포인트 추출
valid_keypoints = {}

for image_file in sorted(os.listdir(image_folder)):
    if image_file.lower().endswith(('.jpg', '.png')):
        image_path = os.path.join(image_folder, image_file)
        image = cv2.imread(image_path)
        results = model(image, verbose=False)

        # 왼쪽 사람 키포인트 추출
        keypoints = filter_keypoints_and_select_leftmost(results[0], image.shape[0])
        if keypoints:
            valid_keypoints[image_file] = keypoints
            print(f"{image_file}: Keypoints Extracted")
        else:
            print(f"{image_file}: No Keypoints")

# 결과 저장
output_file = os.path.join(output_folder, "valid_keypoints_left_only.npy")
np.save(output_file, valid_keypoints)
print("Leftmost keypoints extraction completed and saved.")


### 왼쪽 사람 키포인트 이미지로 추출

In [ ]:
import numpy as np
import os
import cv2

# 경로 설정
keypoints_file = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/filtered_keypoints_left_only/valid_keypoints_left_only.npy"
image_folder = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/aegis_pose_data/violence_B_30fps_flipped"
output_folder = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/keypoint_flipped"
os.makedirs(output_folder, exist_ok=True)

# 키포인트 시각화 함수
def visualize_keypoints(image_path, keypoints, output_path):
    image = cv2.imread(image_path)
    for name, (x, y) in keypoints.items():  # 키포인트 순회
        cv2.circle(image, (x, y), 5, (0, 0, 255), -1)  # 빨간색 점 그리기
        cv2.putText(image, name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)  # 키포인트 이름 표시
    cv2.imwrite(output_path, image)  # 시각화된 이미지 저장

# 키포인트 파일 불러오기
keypoints_data = np.load(keypoints_file, allow_pickle=True).item()

# 이미지 시각화 실행
for image_file, keypoints in keypoints_data.items():
    image_path = os.path.join(image_folder, image_file)
    output_path = os.path.join(output_folder, image_file)
    if os.path.exists(image_path) and keypoints:
        visualize_keypoints(image_path, keypoints, output_path)
        print(f"Visualized: {image_file}")
    else:
        print(f"Skipped: {image_file}")

print(f"Keypoint visualization complete. Images saved in {output_folder}")


### 키포인트 그래프로 시각화

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 경로 설정
keypoints_file = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/filtered_keypoints_left_only/valid_keypoints_left_only.npy"

# 키포인트 데이터 불러오기
keypoints_data = np.load(keypoints_file, allow_pickle=True).item()

# 키포인트 변화량 계산 함수 (존재하던 키포인트만)
def calculate_filtered_changes(data):
    previous_coords = {}
    changes = {key: [] for key in ["left_shoulder", "left_elbow", "left_wrist", "right_shoulder", "right_elbow", "right_wrist"]}
    frame_indices = []

    for frame_index, (frame, keypoints) in enumerate(sorted(data.items())):
        frame_indices.append(frame_index)
        
        for key in changes.keys():
            current_coords = keypoints.get(key, None)  # 현재 프레임 키포인트
            if key in previous_coords and current_coords:  
                # 존재하던 키포인트의 변화량만 계산
                delta = abs(current_coords[0] - previous_coords[key][0]) + abs(current_coords[1] - previous_coords[key][1])
                changes[key].append(delta)
            else:
                # 키포인트가 새로 생성되었거나 누락된 경우 0으로 처리
                changes[key].append(0)
            
            # 이전 키포인트 업데이트 (현재 좌표가 있는 경우만)
            if current_coords:
                previous_coords[key] = current_coords

    return frame_indices, changes

# 변화량 계산
frame_indices, filtered_changes = calculate_filtered_changes(keypoints_data)

# 변화량 시각화 함수
def plot_filtered_changes(frame_indices, changes):
    plt.figure(figsize=(12, 8))

    for key, deltas in changes.items():
        plt.plot(frame_indices, deltas, label=f"{key}_change")

    plt.title("Filtered Keypoint Coordinate Changes Over Frames")
    plt.xlabel("Frame Index")
    plt.ylabel("Coordinate Changes")
    plt.legend()
    plt.grid(True)
    plt.show()

# 그래프 출력
plot_filtered_changes(frame_indices, filtered_changes)


### 왼쪽 손목팔꿈치 변화량 그래프

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 경로 설정
keypoints_file = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/filtered_keypoints_left_only/valid_keypoints_left_only.npy"

# 키포인트 데이터 불러오기
keypoints_data = np.load(keypoints_file, allow_pickle=True).item()

# 왼쪽 손목과 팔꿈치 변화량 계산 함수
def calculate_filtered_changes(data):
    previous_coords = {}
    changes = {"left_elbow": [], "left_wrist": []}  # 왼쪽 팔꿈치와 손목만 고려
    frame_indices = []

    for frame_index, (frame, keypoints) in enumerate(sorted(data.items())):
        frame_indices.append(frame_index)

        for key in changes.keys():  # 왼쪽 팔꿈치와 손목만 순회
            current_coords = keypoints.get(key, None)  # 현재 프레임 키포인트
            if key in previous_coords and current_coords:  
                # 존재하던 키포인트의 변화량만 계산
                delta = abs(current_coords[0] - previous_coords[key][0]) + abs(current_coords[1] - previous_coords[key][1])
                changes[key].append(delta)
            else:
                # 키포인트가 새로 생성되었거나 누락된 경우 0으로 처리
                changes[key].append(0)
            
            # 이전 키포인트 업데이트 (현재 좌표가 있는 경우만)
            if current_coords:
                previous_coords[key] = current_coords

    return frame_indices, changes

# 변화량 계산
frame_indices, filtered_changes = calculate_filtered_changes(keypoints_data)

# 변화량 시각화 함수
def plot_filtered_changes(frame_indices, changes):
    plt.figure(figsize=(12, 8))

    # 왼쪽 팔꿈치와 손목만 시각화
    for key, deltas in changes.items():
        plt.plot(frame_indices, deltas, label=f"{key}_change", linewidth=2)

    plt.title("Left Elbow and Wrist Coordinate Changes Over Frames")
    plt.xlabel("Frame Index")
    plt.ylabel("Coordinate Changes")
    plt.legend()
    plt.grid(True)
    plt.show()

# 그래프 출력
plot_filtered_changes(frame_indices, filtered_changes)


### 오른쪽과 왼쪽 npy 합치기

In [ ]:
import numpy as np

# 경로 설정
keypoints_file_left = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/filtered_keypoints_left_only/valid_keypoints_left_only.npy"
keypoints_file_right = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/filtered_keypoints_right_only/valid_keypoints_right_only.npy"
output_data_file = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/combined_lstm_data_left_right.npy"

# 하이퍼파라미터 설정
SEQUENCE_LENGTH = 10
THRESHOLD_FACTOR = 10
THRESHOLD_RATIO = 0.3  # 시퀀스 내 30% 이상 변화량이 임계값 초과하면 폭력으로 라벨링

# 변화량 계산 함수
def calculate_combined_changes(left_data, right_data):
    combined_data = {}
    previous_coords = {"left": {}, "right": {}}
    sequence = []
    labels = []

    # 모든 프레임 병합
    for frame in sorted(set(left_data.keys()) | set(right_data.keys())):
        frame_changes = []

        # 왼쪽 데이터
        left_keypoints = left_data.get(frame, {})
        for key in ["left_elbow", "left_wrist"]:
            current_coords = left_keypoints.get(key)
            prev_coords = previous_coords["left"].get(key)
            delta = abs(current_coords[0] - prev_coords[0]) + abs(current_coords[1] - prev_coords[1]) if current_coords and prev_coords else 0
            frame_changes.append(delta)
            previous_coords["left"][key] = current_coords

        # 오른쪽 데이터
        right_keypoints = right_data.get(frame, {})
        for key in ["right_elbow", "right_wrist"]:
            current_coords = right_keypoints.get(key)
            prev_coords = previous_coords["right"].get(key)
            delta = abs(current_coords[0] - prev_coords[0]) + abs(current_coords[1] - prev_coords[1]) if current_coords and prev_coords else 0
            frame_changes.append(delta)
            previous_coords["right"][key] = current_coords

        sequence.append(frame_changes)

        # 라벨링
        high_change_frames = sum(1 for val in frame_changes if val >= THRESHOLD_FACTOR)
        label = 1 if high_change_frames / len(frame_changes) >= THRESHOLD_RATIO else 0
        labels.append(label)

    return np.array(sequence), np.array(labels)

# 데이터 불러오기
left_data = np.load(keypoints_file_left, allow_pickle=True).item()
right_data = np.load(keypoints_file_right, allow_pickle=True).item()

# 변화량 데이터 결합
print("Calculating combined coordinate changes...")
X, y = calculate_combined_changes(left_data, right_data)

# 시퀀스 생성
X_sequences, y_sequences = [], []
for i in range(len(X) - SEQUENCE_LENGTH):
    X_sequences.append(X[i:i + SEQUENCE_LENGTH])
    y_sequences.append(max(y[i:i + SEQUENCE_LENGTH]))

X_sequences = np.array(X_sequences)
y_sequences = np.array(y_sequences)

# 데이터 저장
np.save(output_data_file, {"X": X_sequences, "y": y_sequences})
print(f"Combined LSTM data saved to {output_data_file}")
print(f"Shape of X: {X_sequences.shape}, Shape of y: {y_sequences.shape}")


### 저장된 데이터 확인

In [ ]:
import numpy as np

# 저장된 파일 경로
output_data_file = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/coordinate_changes_labeled.npy"

# 데이터 불러오기
data = np.load(output_data_file, allow_pickle=True).item()
X, y = data["X"], data["y"]

# 데이터 형태 확인
print("Shape of X (input sequences):", X.shape)  # 입력 시퀀스의 형태
print("Shape of y (labels):", y.shape)  # 라벨의 형태

# 일부 데이터 내용 확인
print("\nFirst input sequence (X):")
print(X[0])  # 첫 번째 시퀀스 확인

print("\nCorresponding label (y):")
print(y[0])  # 첫 번째 시퀀스의 라벨 확인

# 통계 정보 확인
print("\nStatistics of Input Sequences (X):")
print("Max value:", X.max())
print("Min value:", X.min())
print("Mean value:", X.mean())

#
sequence_index = 0  # 첫 번째 시퀀스 확인
print(f"Frame-by-frame changes for Sequence {sequence_index}:")
print(X[sequence_index])  # 시퀀스 내 각 프레임 변화량



In [39]:
### 변화량데이터생성및 라벨링

In [ ]:
import numpy as np

# 경로 설정
keypoints_file_left = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/filtered_keypoints_left_only/valid_keypoints_left_only.npy"
keypoints_file_right = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/filtered_keypoints_right_only/valid_keypoints_right_only.npy"
output_data_file = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/labeled_lstm_data_with_ratio_combined_left_right.npy"

# 하이퍼파라미터 설정
SEQUENCE_LENGTH = 10  # LSTM 입력 시퀀스 길이
THRESHOLD_FACTOR = 10  # 변화량 임계값
THRESHOLD_RATIO = 0.3  # 시퀀스 내 30% 이상 프레임이 임계값을 넘으면 폭력으로 인식

# 데이터 준비 함수
def prepare_data_with_ratio(data):
    previous_coords = {}
    sequence = []
    labels = []

    # 프레임별 변화량 계산
    for frame, keypoints in sorted(data.items()):
        frame_changes = []
        for key in ["right_wrist", "right_elbow", "left_wrist", "left_elbow"]:
            current_coords = keypoints.get(key, None)
            if key in previous_coords and current_coords:
                # 변화량 계산
                delta = abs(current_coords[0] - previous_coords[key][0]) + abs(current_coords[1] - previous_coords[key][1])
                frame_changes.append(delta)
            else:
                frame_changes.append(0)  # 키포인트가 없으면 변화량 0으로 설정
            if current_coords:
                previous_coords[key] = current_coords  # 이전 프레임 업데이트
        sequence.append(frame_changes)

    # 시퀀스 생성 및 라벨링
    X, y = [], []
    for i in range(len(sequence) - SEQUENCE_LENGTH):
        sub_sequence = sequence[i:i+SEQUENCE_LENGTH]  # 시퀀스 길이만큼 추출
        high_change_frames = sum(1 for frame in sub_sequence if max(frame) >= THRESHOLD_FACTOR)
        label = 1 if high_change_frames / SEQUENCE_LENGTH >= THRESHOLD_RATIO else 0
        X.append(sub_sequence)
        y.append(label)

    return np.array(X), np.array(y)

# 키포인트 데이터 불러오기
def load_keypoints_data(file_path):
    print(f"Loading keypoints data from {file_path}...")
    data = np.load(file_path, allow_pickle=True)
    if isinstance(data, np.ndarray):  # numpy 배열인 경우
        data = data.item()
    return data

# 왼쪽과 오른쪽 키포인트 데이터 불러오기 및 결합
keypoints_data_left = load_keypoints_data(keypoints_file_left)
keypoints_data_right = load_keypoints_data(keypoints_file_right)
combined_keypoints_data = {**keypoints_data_left, **keypoints_data_right}

# 변화량 데이터 준비
print("Calculating coordinate changes and labeling data...")
X, y = prepare_data_with_ratio(combined_keypoints_data)

# 결과 저장
print(f"Saving labeled data to {output_data_file}...")
np.save(output_data_file, {"X": X, "y": y})

# 데이터 확인
print("Data preparation complete.")
print(f"Shape of X (input sequences): {X.shape}")
print(f"Shape of y (labels): {y.shape}")
print(f"Example sequence:\n{X[0]}")
print(f"Corresponding label: {y[0]}")


In [41]:
## 모든시퀀스 변화량

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 경로 설정
data_file = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/combined_lstm_data_left_right.npy"

# 데이터 로드
data = np.load(data_file, allow_pickle=True).item()
X, y = data["X"], data["y"]

# 데이터 정보 출력
print("Shape of X (input sequences):", X.shape)  # (시퀀스 수, 시퀀스 길이, 키포인트 수)
print("Shape of y (labels):", y.shape)          # (시퀀스 수,)
print(f"Number of Violent (Label=1): {np.sum(y)}")
print(f"Number of Non-Violent (Label=0): {len(y) - np.sum(y)}")

# 시퀀스 평균 변화량 계산 (시퀀스 단위로 압축)
average_changes = X.mean(axis=1)  # 각 시퀀스에 대해 키포인트별 평균 변화량

# 전체 시퀀스 평균 변화량 시각화
def plot_all_sequences(average_changes, labels):
    plt.figure(figsize=(12, 8))
    for i in range(len(average_changes)):
        color = 'red' if labels[i] == 1 else 'blue'  # 폭력(1)은 빨간색, 비폭력(0)은 파란색
        plt.plot(range(len(average_changes[i])), average_changes[i], color=color, alpha=0.5)

    plt.title("Average Keypoint Changes for All Sequences")
    plt.xlabel("Frame Index in Sequence")
    plt.ylabel("Coordinate Changes")
    plt.grid(True)
    plt.show()

# 전체 시퀀스 시각화
plot_all_sequences(average_changes, y)


In [43]:
### 시퀀스별 평균 변화량 시각화

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 경로 설정
data_file = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/combined_lstm_data_left_right.npy"
# 데이터 로드
data = np.load(data_file, allow_pickle=True).item()
X, y = data["X"], data["y"]

# 데이터 정보 출력
print("Shape of X (input sequences):", X.shape)  # (시퀀스 수, 시퀀스 길이, 키포인트 수)
print("Shape of y (labels):", y.shape)          # (시퀀스 수,)
print(f"Number of Sequences: {len(X)}")
print(f"Number of Violent (Label=1): {np.sum(y)}")
print(f"Number of Non-Violent (Label=0): {len(y) - np.sum(y)}")

# 시퀀스별 평균 변화량 계산
average_changes = X.mean(axis=(1, 2))  # 시퀀스 단위로 평균 변화량 계산

# 시퀀스별 평균 변화량 시각화
def plot_sequence_changes(average_changes, labels):
    plt.figure(figsize=(12, 6))

    # 시퀀스별 평균 변화량 그리기
    for i, (change, label) in enumerate(zip(average_changes, labels)):
        color = 'red' if label == 1 else 'blue'  # 폭력(1): 빨간색, 비폭력(0): 파란색
        plt.scatter(i, change, color=color, s=10)  # 각 시퀀스의 평균 변화량

    plt.title("Average Coordinate Changes for Each Sequence")
    plt.xlabel("Sequence Index")
    plt.ylabel("Average Coordinate Changes")
    plt.grid(True)
    plt.legend(["Violent (Label=1)", "Non-Violent (Label=0)"], loc="upper right")
    plt.show()

# 시퀀스별 변화량 시각화
plot_sequence_changes(average_changes, y)


### 변화량 80 ??

In [ ]:
import numpy as np

# 경로 설정
keypoints_file = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/filtered_keypoints_with_left_fixed.npy"
output_data_file = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/labeled_lstm_data_80threshold.npy"

# 파라미터 설정
SEQUENCE_LENGTH = 10  # 시퀀스 길이
THRESHOLD_FACTOR = 80  # 변화량이 80 이상인 경우 폭력으로 감지

# 데이터 준비 함수
def prepare_data_with_new_threshold(data):
    previous_coords = {}
    sequence = []
    labels = []

    # 프레임별 변화량 계산
    for frame, keypoints in sorted(data.items()):
        frame_changes = []
        for key in ["right_wrist", "right_elbow", "left_wrist", "left_elbow"]:
            current_coords = keypoints.get(key, None)
            if key in previous_coords and current_coords:
                # 변화량 계산
                delta = abs(current_coords[0] - previous_coords[key][0]) + abs(current_coords[1] - previous_coords[key][1])
                frame_changes.append(delta)
            else:
                frame_changes.append(0)  # 키포인트가 없으면 변화량 0으로 설정
            if current_coords:
                previous_coords[key] = current_coords  # 이전 프레임 업데이트
        sequence.append(frame_changes)

    # 시퀀스 생성 및 라벨링
    X, y = [], []
    for i in range(len(sequence) - SEQUENCE_LENGTH):
        sub_sequence = sequence[i:i+SEQUENCE_LENGTH]  # 시퀀스 길이만큼 추출
        high_change = any(max(frame) >= THRESHOLD_FACTOR for frame in sub_sequence)  # 80 이상 변화량 존재 여부
        label = 1 if high_change else 0
        X.append(sub_sequence)
        y.append(label)

    return np.array(X), np.array(y)

# 키포인트 데이터 불러오기
print("Loading keypoints data...")
keypoints_data = np.load(keypoints_file, allow_pickle=True).item()

# 변화량 데이터 준비
print("Calculating coordinate changes with new threshold...")
X, y = prepare_data_with_new_threshold(keypoints_data)

# 결과 저장
print(f"Saving relabeled data to {output_data_file}...")
np.save(output_data_file, {"X": X, "y": y})

# 데이터 확인
print("Data preparation complete.")
print(f"Shape of X (input sequences): {X.shape}")
print(f"Shape of y (labels): {y.shape}")
print(f"Number of Violent (Label=1): {np.sum(y)}")
print(f"Number of Non-Violent (Label=0): {len(y) - np.sum(y)}")


In [46]:
### 80 시각화

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 경로 설정
data_file = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/labeled_lstm_data_80threshold.npy"

# 데이터 불러오기
data = np.load(data_file, allow_pickle=True).item()
X, y = data["X"], data["y"]

# 데이터 정보 출력
print("Shape of X (input sequences):", X.shape)  # (시퀀스 수, 시퀀스 길이, 키포인트 수)
print("Shape of y (labels):", y.shape)          # (시퀀스 수,)
print(f"Number of Violent (Label=1): {np.sum(y)}")
print(f"Number of Non-Violent (Label=0): {len(y) - np.sum(y)}")

# 시퀀스별 평균 변화량 계산
average_changes = X.mean(axis=(1, 2))  # 시퀀스 단위로 평균 변화량 계산

# 시각화 함수
def plot_labeled_sequences(average_changes, labels):
    plt.figure(figsize=(12, 6))
    
    for i, (change, label) in enumerate(zip(average_changes, labels)):
        color = 'red' if label == 1 else 'blue'  # 폭력(1)은 빨간색, 비폭력(0)은 파란색
        plt.scatter(i, change, color=color, s=10)

    plt.title("Average Coordinate Changes for Each Sequence (New Threshold)")
    plt.xlabel("Sequence Index")
    plt.ylabel("Average Coordinate Changes")
    plt.grid(True)
    plt.legend(["Violent (Label=1)", "Non-Violent (Label=0)"], loc="upper right")
    plt.show()

# 그래프 시각화
plot_labeled_sequences(average_changes, y)


In [49]:
### 80이상.....

In [ ]:
import numpy as np

# 경로 설정
keypoints_file = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/filtered_keypoints_with_left_fixed.npy"
output_data_file = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/labeled_lstm_data_avg80.npy"

# 파라미터 설정
SEQUENCE_LENGTH = 10  # 시퀀스 길이
THRESHOLD_FACTOR = 80  # 평균 변화량이 80 이상인 경우 폭력으로 라벨링

# 데이터 준비 함수
def prepare_data_with_avg_threshold(data):
    previous_coords = {}
    sequence = []

    # 프레임별 변화량 계산
    for frame, keypoints in sorted(data.items()):
        frame_changes = []
        for key in ["right_wrist", "right_elbow", "left_wrist", "left_elbow"]:
            current_coords = keypoints.get(key, None)
            if key in previous_coords and current_coords:
                # 변화량 계산
                delta = abs(current_coords[0] - previous_coords[key][0]) + abs(current_coords[1] - previous_coords[key][1])
                frame_changes.append(delta)
            else:
                frame_changes.append(0)  # 키포인트가 없으면 변화량 0
            if current_coords:
                previous_coords[key] = current_coords
        sequence.append(frame_changes)

    # 시퀀스 생성 및 라벨링
    X, y = [], []
    for i in range(len(sequence) - SEQUENCE_LENGTH):
        sub_sequence = sequence[i:i+SEQUENCE_LENGTH]
        avg_change = np.mean(sub_sequence)  # 시퀀스 평균 변화량 계산
        label = 1 if avg_change >= THRESHOLD_FACTOR else 0  # 평균 변화량이 80 이상인 경우 라벨 1
        X.append(sub_sequence)
        y.append(label)

    return np.array(X), np.array(y)

# 키포인트 데이터 불러오기
print("Loading keypoints data...")
keypoints_data = np.load(keypoints_file, allow_pickle=True).item()

# 변화량 데이터 준비
print("Calculating coordinate changes with average threshold...")
X, y = prepare_data_with_avg_threshold(keypoints_data)

# 결과 저장
print(f"Saving relabeled data to {output_data_file}...")
np.save(output_data_file, {"X": X, "y": y})

# 데이터 확인
print("Data preparation complete.")
print(f"Shape of X (input sequences): {X.shape}")
print(f"Shape of y (labels): {y.shape}")
print(f"Number of Violent (Label=1): {np.sum(y)}")
print(f"Number of Non-Violent (Label=0): {len(y) - np.sum(y)}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 경로 설정
data_file = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/labeled_lstm_data_avg80.npy"

# 데이터 불러오기
data = np.load(data_file, allow_pickle=True).item()
X, y = data["X"], data["y"]

# 데이터 정보 출력
print("Shape of X (input sequences):", X.shape)  # (시퀀스 수, 시퀀스 길이, 키포인트 수)
print("Shape of y (labels):", y.shape)          # (시퀀스 수,)
print(f"Number of Violent (Label=1): {np.sum(y)}")
print(f"Number of Non-Violent (Label=0): {len(y) - np.sum(y)}")

# 시퀀스별 평균 변화량 계산
average_changes = X.mean(axis=(1, 2))  # 시퀀스 단위로 평균 변화량 계산

# 시각화 함수
def plot_labeled_sequences(average_changes, labels):
    plt.figure(figsize=(12, 6))
    
    for i, (change, label) in enumerate(zip(average_changes, labels)):
        color = 'red' if label == 1 else 'blue'  # 폭력(1)은 빨간색, 비폭력(0)은 파란색
        plt.scatter(i, change, color=color, s=10)

    plt.title("Average Coordinate Changes for Each Sequence (Updated Threshold)")
    plt.xlabel("Sequence Index")
    plt.ylabel("Average Coordinate Changes")
    plt.grid(True)
    plt.legend(["Violent (Label=1)", "Non-Violent (Label=0)"], loc="upper right")
    plt.show()

# 그래프 시각화
plot_labeled_sequences(average_changes, y)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 경로 설정
data_file = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/combined_lstm_data_left_right.npy"
output_file = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/relabeled_lstm_data_8threshold.npy"

# 데이터 로드
data = np.load(data_file, allow_pickle=True).item()
X, y = data["X"], data["y"]

# 데이터 정보 출력
print("Shape of X (input sequences):", X.shape)
print("Shape of y (labels):", y.shape)
print(f"Number of Sequences: {len(X)}")

# 시퀀스별 평균 변화량 계산
average_changes = X.mean(axis=(1, 2))  # 시퀀스 단위로 평균 변화량 계산

# 평균 변화량 기반으로 새로운 라벨링 (6 이상만 폭력으로)
new_labels = np.array([1 if change > 6 else 0 for change in average_changes])

# 새 라벨링된 데이터 저장
np.save(output_file, {"X": X, "y": new_labels})
print(f"Relabeled data saved to {output_file}")

# 시퀀스별 평균 변화량 시각화
def plot_sequence_changes(average_changes, labels):
    plt.figure(figsize=(12, 6))

    # 시퀀스별 평균 변화량 그리기
    for i, (change, label) in enumerate(zip(average_changes, labels)):
        color = 'red' if label == 1 else 'blue'  # 폭력(1): 빨간색, 비폭력(0): 파란색
        plt.scatter(i, change, color=color, s=10)  # 각 시퀀스의 평균 변화량

    plt.title("Average Coordinate Changes for Each Sequence (Updated Threshold)")
    plt.xlabel("Sequence Index")
    plt.ylabel("Average Coordinate Changes")
    plt.grid(True)
    plt.legend(["Violent (Label=1)", "Non-Violent (Label=0)"], loc="upper right")
    plt.show()

# 새로운 라벨링으로 시각화
plot_sequence_changes(average_changes, new_labels)

# 결과 확인
print("Updated Label Distribution:")
print(f"Number of Violent (Label=1): {np.sum(new_labels)}")
print(f"Number of Non-Violent (Label=0): {len(new_labels) - np.sum(new_labels)}")


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim

# 데이터 로드
data_file = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/relabeled_lstm_data_8threshold.npy"
data = np.load(data_file, allow_pickle=True).item()
X, y = data["X"], data["y"]

# 데이터 전처리: 학습/테스트 나누기
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_test = torch.tensor(X_train, dtype=torch.float32), torch.tensor(X_test, dtype=torch.float32)
y_train, y_test = torch.tensor(y_train, dtype=torch.long), torch.tensor(y_test, dtype=torch.long)

# LSTM 모델 정의
class LSTMModel(nn.Module):
    def __init__(self, input_size=4, hidden_size=64, num_layers=2, output_size=2):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        _, (hn, _) = self.lstm(x)
        out = self.fc(hn[-1])  # 마지막 LSTM 레이어의 hidden state 사용
        return out

# 모델 초기화
model = LSTMModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 모델 학습
epochs = 30
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

# 모델 평가
model.eval()
with torch.no_grad():
    y_pred = model(X_test)
    y_pred_labels = torch.argmax(y_pred, dim=1)
    accuracy = (y_pred_labels == y_test).sum().item() / len(y_test)
    print(f"Test Accuracy: {accuracy:.4f}")


### 평가 !!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# 혼동 행렬 생성
conf_matrix = confusion_matrix(y_test, y_pred_labels)

# 혼동 행렬 시각화
plt.figure(figsize=(6, 5))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap="Blues", xticklabels=["Non-Violent", "Violent"], yticklabels=["Non-Violent", "Violent"])
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

# 정밀도, 재현율, F1-Score 출력
print("Classification Report:")
print(classification_report(y_test, y_pred_labels, target_names=["Non-Violent", "Violent"]))


In [ ]:
# 모델 저장
torch.save(model.state_dict(), "violence_detection_lstm_left_right.pth")
print("Model saved successfully!")


In [ ]:
import os
print("Current working directory:", os.getcwd())


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Flowchart steps in English
steps_flowchart_eng = [
    {"name": "Load YOLOv8 Model & Extract Keypoints", "xy": (0.5, 6)},
    {"name": "Filter Keypoints & Save Data", "xy": (0.5, 5)},
    {"name": "Calculate Coordinate Changes", "xy": (0.5, 4)},
    {"name": "Generate Sequence Data", "xy": (0.5, 3)},
    {"name": "Prepare Data & Train LSTM", "xy": (0.5, 2)},
    {"name": "Real-time Inference & End", "xy": (0.5, 1)},
]

# Create the flowchart
fig, ax = plt.subplots(figsize=(10, 8))

# Draw rectangles and add text for each step
for step in steps_flowchart_eng:
    rect = patches.Rectangle((step["xy"][0] - 0.4, step["xy"][1] - 0.4), 0.8, 0.8, edgecolor='black', facecolor='lightblue')
    ax.add_patch(rect)
    ax.text(step["xy"][0], step["xy"][1], step["name"], ha='center', va='center', fontsize=10)

# Add arrows between steps
for i in range(len(steps_flowchart_eng) - 1):
    ax.annotate("", xy=steps_flowchart_eng[i+1]["xy"], xytext=steps_flowchart_eng[i]["xy"],
                arrowprops=dict(facecolor='black', shrink=0.05, width=1, headwidth=8))

# Set axis limits and hide axes
ax.set_xlim(0, 1)
ax.set_ylim(0, 7)
ax.axis('off')

# Title for the flowchart
plt.title("YOLOv8 and LSTM-Based Code Flowchart", fontsize=14)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Define the steps in the system flowchart with fully English terms
steps = [
    {"name": "Start Sender (A_Sender)", "xy": (1, 8)},
    {"name": "Capture Video Frames\nfrom Cameras", "xy": (1, 7)},
    {"name": "Stream Frames to:\nFire Station, Police Station,\nCentral Server", "xy": (1, 6)},
    {"name": "Fire Station\n(B_FireStation)", "xy": (0, 5)},
    {"name": "Police Station\n(C_PoliceStation)", "xy": (2, 5)},
    {"name": "Receive Frames", "xy": (0, 4)},
    {"name": "Receive Frames", "xy": (2, 4)},
    {"name": "Detect Fire\n(YOLO)", "xy": (0, 3)},
    {"name": "Detect Weapons\n(YOLO)", "xy": (2, 3)},
    {"name": "Send Fire Labels\nto Central Server", "xy": (0, 2)},
    {"name": "Send Weapon Labels\nto Central Server", "xy": (2, 2)},
    {"name": "Central Server\n(D_CentralServer)", "xy": (1, 1)},
    {"name": "Combine Frames\n& Labels", "xy": (1, 0)},
    {"name": "Save Metadata to\nDatabase", "xy": (1, -1)},
    {"name": "Provide Video Streaming\nvia Flask", "xy": (1, -2)},
]

# Arrows connecting the steps, adjusted to avoid overlapping text
arrows = [
    ((1, 7.8), (1, 7.2)),  # Start Sender -> Capture Video Frames
    ((1, 6.8), (1, 6.2)),  # Capture Video Frames -> Stream Frames
    ((0.8, 6), (0.2, 5.4)),  # Stream Frames -> Fire Station
    ((1.2, 6), (1.8, 5.4)),  # Stream Frames -> Police Station
    ((0, 4.8), (0, 4.2)),  # Fire Station -> Receive Frames
    ((2, 4.8), (2, 4.2)),  # Police Station -> Receive Frames
    ((0, 3.8), (0, 3.2)),  # Receive Frames -> Detect Fire
    ((2, 3.8), (2, 3.2)),  # Receive Frames -> Detect Weapons
    ((0, 2.8), (0, 2.2)),  # Detect Fire -> Send Fire Labels
    ((2, 2.8), (2, 2.2)),  # Detect Weapons -> Send Weapon Labels
    ((0.2, 2), (0.8, 1.4)),  # Send Fire Labels -> Central Server
    ((1.8, 2), (1.2, 1.4)),  # Send Weapon Labels -> Central Server
    ((1, 0.8), (1, 0.2)),  # Central Server -> Combine Frames
    ((1, -0.2), (1, -0.8)),  # Combine Frames -> Save Metadata
    ((1, -1.2), (1, -1.8)),  # Save Metadata -> Provide Streaming
]

# Create the flowchart
fig, ax = plt.subplots(figsize=(12, 12))

# Draw rectangles for each step
for step in steps:
    rect = patches.Rectangle((step["xy"][0] - 0.5, step["xy"][1] - 0.4), 1, 0.8, edgecolor='black', facecolor='lightblue')
    ax.add_patch(rect)
    ax.text(step["xy"][0], step["xy"][1], step["name"], ha='center', va='center', fontsize=10)

# Draw arrows connecting the steps
for start, end in arrows:
    ax.annotate("", xy=end, xytext=start, arrowprops=dict(facecolor='black', shrink=0.05, width=1, headwidth=8))

# Set axis limits and hide axes
ax.set_xlim(-1, 3)
ax.set_ylim(-3, 9)
ax.axis('off')

# Title of the flowchart
plt.title("Cord Flow Chart", fontsize=14)
plt.show()
